In [1]:
import requests
from html import unescape
import re
from bs4 import BeautifulSoup

In [2]:
url = 'https://www.bbc.com/sitemaps/https-index-com-news.xml'

In [3]:
res = requests.get(url)
if res.status_code != 200:
    print('ohho')

soup = BeautifulSoup(res.text, 'xml')

sitemap_tags = soup.find_all('sitemap')

sitemaps = []
for sitemap in sitemap_tags:
    loc = sitemap.find('loc')
    sitemaps.append(loc.text)

urls = []
regex_1 = r"^https://www.bbc.com/news/.*$"
regex_2 = r"^https://www.bbc.com/sport/.*$"
for sitemap in sitemaps:
    response = requests.get(sitemap)
    soup = BeautifulSoup(response.text, 'xml')
    url_tags = soup.find_all('url')
    
    for url in url_tags:
        loc = url.find('loc')
        if re.match(regex_1, loc.text) or re.match(regex_2, loc.text):
            urls.append(loc.text)

In [5]:
urls

['https://www.bbc.com/sport/articles/cv2pqkvz2v2o',
 'https://www.bbc.com/sport/football/64815400',
 'https://www.bbc.com/news/uk-england-shropshire-64831704',
 'https://www.bbc.com/news/uk-england-somerset-64784732',
 'https://www.bbc.com/news/world-asia-china-64807507',
 'https://www.bbc.com/news/uk-politics-64823708',
 'https://www.bbc.com/news/world-europe-jersey-64827771',
 'https://www.bbc.com/news/uk-england-wiltshire-64830604',
 'https://www.bbc.com/sport/football/64809663',
 'https://www.bbc.com/sport/articles/c0dl23pv98go',
 'https://www.bbc.com/sport/golf/64815156',
 'https://www.bbc.com/news/uk-england-london-64809875',
 'https://www.bbc.com/news/uk-england-birmingham-64825372',
 'https://www.bbc.com/news/uk-england-merseyside-64833475',
 'https://www.bbc.com/news/uk-scotland-scotland-politics-64809925',
 'https://www.bbc.com/news/uk-england-south-yorkshire-64814814',
 'https://www.bbc.com/news/uk-england-hampshire-64814304',
 'https://www.bbc.com/news/uk-england-nottingham

In [175]:
visited = set()

In [182]:
def url_to_text(url):
    try:
        response = requests.get(url)
        if response.status_code != 200:
            return None
        
        soup = BeautifulSoup(response.text, 'html.parser')
        full_article = ''
        text = []
        heading = soup.select('h1')[0].text.strip()
        text.append(heading)
        
        for p in soup.select('p'):
            text.append(unescape(p.text))
        full_article = ' '.join(text)
        
        return (url, full_article) 
    except requests.exceptions.Timeout as ee :
        return None


In [188]:
import psycopg2

# Connect to the PostgreSQL database
conn = psycopg2.connect(
    host="localhost",
    database="SearchEngine",
    user="postgres",
    password="2580"
)

# Create a cursor object
cursor = conn.cursor()

# Open a cursor to perform database operations
cur = conn.cursor()


for url in urls:
    try:
        url, text = url_to_text(url)
        data = (url, text)
        sql = sql = "INSERT INTO sites (url, content) VALUES (%s, %s)"
        cur.execute(sql, data)
        conn.commit()
    except TypeError as ee :
        continue


 Error was bypassed
 Error was bypassed
 Error was bypassed
 Error was bypassed
 Error was bypassed
 Error was bypassed
 Error was bypassed
 Error was bypassed
 Error was bypassed
 Error was bypassed
 Error was bypassed
 Error was bypassed
 Error was bypassed
 Error was bypassed
 Error was bypassed
 Error was bypassed
 Error was bypassed
 Error was bypassed
 Error was bypassed
 Error was bypassed
 Error was bypassed
 Error was bypassed
 Error was bypassed
 Error was bypassed
 Error was bypassed
 Error was bypassed
 Error was bypassed
 Error was bypassed
 Error was bypassed
 Error was bypassed
 Error was bypassed
 Error was bypassed
 Error was bypassed
 Error was bypassed
 Error was bypassed
 Error was bypassed
 Error was bypassed
 Error was bypassed
 Error was bypassed
 Error was bypassed
 Error was bypassed
 Error was bypassed
 Error was bypassed
 Error was bypassed
 Error was bypassed
 Error was bypassed
 Error was bypassed
 Error was bypassed
 Error was bypassed
 Error was bypassed


KeyboardInterrupt: 

In [ ]:
cur.close()
conn.close()